## examine_functions.ipynb
### WESmith 05/09/20
- look at the individual node functions using truth tables

In [48]:
import phial.experiment as ws
import phial.toolbox  as tb
from phial.utils import tic,toc
import itertools as it
import pandas as pd
#tic() # start tracking time of Notebook load

In [50]:
def test_func(fun, N):
    return pd.DataFrame([(k, (fun(k))) for k in it.product([0,1], repeat=N)])

In [51]:
for k in [2,3,4]:
    print (test_func(tb.xor_func, k))

        0  1
0  (0, 0)  0
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  0
           0  1
0  (0, 0, 0)  0
1  (0, 0, 1)  1
2  (0, 1, 0)  1
3  (0, 1, 1)  0
4  (1, 0, 0)  1
5  (1, 0, 1)  0
6  (1, 1, 0)  0
7  (1, 1, 1)  1
               0  1
0   (0, 0, 0, 0)  0
1   (0, 0, 0, 1)  1
2   (0, 0, 1, 0)  1
3   (0, 0, 1, 1)  0
4   (0, 1, 0, 0)  1
5   (0, 1, 0, 1)  0
6   (0, 1, 1, 0)  0
7   (0, 1, 1, 1)  1
8   (1, 0, 0, 0)  1
9   (1, 0, 0, 1)  0
10  (1, 0, 1, 0)  0
11  (1, 0, 1, 1)  1
12  (1, 1, 0, 0)  0
13  (1, 1, 0, 1)  1
14  (1, 1, 1, 0)  1
15  (1, 1, 1, 1)  0


In [52]:
for k in [2,3,4]:
    print (test_func(tb.or_func, k))

        0  1
0  (0, 0)  0
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  1
           0  1
0  (0, 0, 0)  0
1  (0, 0, 1)  1
2  (0, 1, 0)  1
3  (0, 1, 1)  1
4  (1, 0, 0)  1
5  (1, 0, 1)  1
6  (1, 1, 0)  1
7  (1, 1, 1)  1
               0  1
0   (0, 0, 0, 0)  0
1   (0, 0, 0, 1)  1
2   (0, 0, 1, 0)  1
3   (0, 0, 1, 1)  1
4   (0, 1, 0, 0)  1
5   (0, 1, 0, 1)  1
6   (0, 1, 1, 0)  1
7   (0, 1, 1, 1)  1
8   (1, 0, 0, 0)  1
9   (1, 0, 0, 1)  1
10  (1, 0, 1, 0)  1
11  (1, 0, 1, 1)  1
12  (1, 1, 0, 0)  1
13  (1, 1, 0, 1)  1
14  (1, 1, 1, 0)  1
15  (1, 1, 1, 1)  1


In [53]:
for k in [2,3,4]:
    print (test_func(tb.and_func, k))

        0  1
0  (0, 0)  0
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  1
           0  1
0  (0, 0, 0)  0
1  (0, 0, 1)  0
2  (0, 1, 0)  0
3  (0, 1, 1)  0
4  (1, 0, 0)  0
5  (1, 0, 1)  0
6  (1, 1, 0)  0
7  (1, 1, 1)  1
               0  1
0   (0, 0, 0, 0)  0
1   (0, 0, 0, 1)  0
2   (0, 0, 1, 0)  0
3   (0, 0, 1, 1)  0
4   (0, 1, 0, 0)  0
5   (0, 1, 0, 1)  0
6   (0, 1, 1, 0)  0
7   (0, 1, 1, 1)  0
8   (1, 0, 0, 0)  0
9   (1, 0, 0, 1)  0
10  (1, 0, 1, 0)  0
11  (1, 0, 1, 1)  0
12  (1, 1, 0, 0)  0
13  (1, 1, 0, 1)  0
14  (1, 1, 1, 0)  0
15  (1, 1, 1, 1)  1
